In [1]:
import pyspark.sql.functions as f

In [2]:
from pyspark.sql import SparkSession

if not 'spark' in locals():
    spark = SparkSession.builder \
        .master("local[*]") \
        .config("spark.driver.memory","4G") \
        .getOrCreate()

spark

In [3]:
spark.conf.set("spark.sql.shuffle.partitions", 16)

# 1. Connect to data source

First you need to fill a Kafka topic, for example via

    s3cat.py -I1 -B10 s3://dimajix-training/data/twitter-sample/ | /opt/kafka/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic twitter

Then we connect to the raw data socket as the datasource by using the `DataStreamReader` API via `spark.readStream`. We need to specify the options `kafka.bootstrap.servers` and `subscribe` and we need to use the format `kafka` for connecting to the data source. The Kafka topic will stream Twitter data samples in raw JSON format, i.e. one JSON document per line.

In [4]:
!hostname

ip-10-200-1-167


In [5]:
# Fill in the correct AWS VPC address of your master host
master = "ip-10-200-1-167:9092"

In [6]:
# Connect to Kafka using the DataStreamReader API via spark.readStream. You need to specify the options `kafka.bootstrap.servers`, `subscribe` and you need to use the format `kafka`
lines = spark.readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", master) \
  .option("subscribe", "twitter") \
  .option("startingOffsets", "latest") \
  .load()

## 1.1 Inspect Schema

The result of the load method is a `DataFrame` again, but a streaming one. This `DataFrame` again has a schema, which we can inspect with the usual method:

In [7]:
lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



# 2. Inspect Data

Of course we also want to inspect the data inside the DataFrame. But this time, we cannot simply invoke `show`, because normal actions do not (directly) work on streaming DataFrames. Instead we need to create a continiuous query. Later, we will see a neat trick how a streaming query can be transformed into a volatile table.

In order to create a continuous query, we need to perform the following steps

1. Create a `DataStreamWriter` by using the `writeStream` method of a DataFrame
2. Specify the output format. We use `console` in our case
3. Specify a checkpoint location on HDFS. This is required for restarting
4. Optionally specify a processing period
5. Start the query

In [8]:
import time



query = lines \
    .withColumn("value", lines["value"].cast("string")) \
    .writeStream \
    .format("console") \
    .outputMode("append") \
    .option("truncate", True) \
    .option("checkpointLocation", "/tmp/zeppelin/checkpoint-twitter-print-" + str(time.time())) \
    .start()

23/11/02 18:32:07 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/11/02 18:32:08 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


## 2.1 Stop Query

In contrast to the RDD API, we can simply stop an individual query instead of a whole StreamingContext by simply calling the `stop` method on the query object. This makes working with streams much easier.

In [9]:
query.stop()

23/11/02 18:32:09 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 0, writer: ConsoleWriter[numRows=20, truncate=true]] is aborting.
23/11/02 18:32:09 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 0, writer: ConsoleWriter[numRows=20, truncate=true]] aborted.


# 3. Counting Hash-Tags

So we now want to create a streaming hashtag count. First we need to extract the Tweet itself from the JSON document, then we need to extract the hashtags in a similar way to the batch word traditional DataFrame word count example, i.e. we split every line into words, keep only hash-tags, group the words and count the sizes of the groups.

Each query looks as follows

```
{ "contributors" : null,
  "coordinates" : null,
  "created_at" : "Fri Jul 29 12:46:00 +0000 2016",
  "entities" : { "hashtags" : [  ],
      "symbols" : [  ],
      "urls" : [ { "display_url" : "fb.me/ItnwZEhy",
            "expanded_url" : "http://fb.me/ItnwZEhy",
            "indices" : [ 33,
                56
              ],
            "url" : "https://t.co/mM0if95F1K"
          } ],
      "user_mentions" : [  ]
    },
  "favorite_count" : 0,
  "favorited" : false,
  "filter_level" : "low",
  "geo" : null,
  "id" : 759007065155117058,
  "id_str" : "759007065155117058",
  "in_reply_to_screen_name" : null,
  "in_reply_to_status_id" : null,
  "in_reply_to_status_id_str" : null,
  "in_reply_to_user_id" : null,
  "in_reply_to_user_id_str" : null,
  "is_quote_status" : false,
  "lang" : "en",
  "place" : null,
  "possibly_sensitive" : false,
  "retweet_count" : 0,
  "retweeted" : false,
  "source" : "<a href=\"http://www.facebook.com/twitter\" rel=\"nofollow\">Facebook</a>",
  "text" : "I posted a new video to Facebook https://t.co/mM0if95F1K",
  "timestamp_ms" : "1469796360659",
  "truncated" : false,
  "user" : { "contributors_enabled" : false,
      "created_at" : "Sat Sep 08 08:28:55 +0000 2012",
      "default_profile" : false,
      "default_profile_image" : false,
      "description" : null,
      "favourites_count" : 0,
      "follow_request_sent" : null,
      "followers_count" : 0,
      "following" : null,
      "friends_count" : 0,
      "geo_enabled" : false,
      "id" : 810489374,
      "id_str" : "810489374",
      "is_translator" : false,
      "lang" : "zh-tw",
      "listed_count" : 0,
      "location" : null,
      "name" : "張冥閻",
      "notifications" : null,
      "profile_background_color" : "FFF04D",
      "profile_background_image_url" : "http://abs.twimg.com/images/themes/theme19/bg.gif",
      "profile_background_image_url_https" : "https://abs.twimg.com/images/themes/theme19/bg.gif",
      "profile_background_tile" : false,
      "profile_image_url" : "http://pbs.twimg.com/profile_images/378800000157469481/0a267258c8ccd1bf53d01c115677dbd7_normal.jpeg",
      "profile_image_url_https" : "https://pbs.twimg.com/profile_images/378800000157469481/0a267258c8ccd1bf53d01c115677dbd7_normal.jpeg",
      "profile_link_color" : "0099CC",
      "profile_sidebar_border_color" : "FFF8AD",
      "profile_sidebar_fill_color" : "F6FFD1",
      "profile_text_color" : "333333",
      "profile_use_background_image" : true,
      "protected" : false,
      "screen_name" : "nineemperor1",
      "statuses_count" : 9652,
      "time_zone" : null,
      "url" : null,
      "utc_offset" : null,
      "verified" : false
    }
}
```

In order to extract a field from a JSON document, we can use the `get_json_object` function.

## 3.1 Extract Tweet

First we need to extract the tweet text itself via the `get_json_object` function and store it into a new column.

In [10]:
ts_text = lines.select(
        lines["timestamp"],
        f.get_json_object(lines["value"].cast("string"), "$.text").alias("text")
    )

In [11]:
ts_text.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- text: string (nullable = true)



## 3.2 Extract Topics

Now that we have the Tweet text itself, we extract all topics with the following approach:
1. Split text along spaces using `split`
2. Create multiple records from all words using `explode`
3. Filter all hash-tags (words that start with a `#`)
4. Filter out all empty topics (topic name only consists of hash-tag `#` itself)

In [12]:
topics = ts_text.select(
        ts_text["timestamp"],
        f.explode(f.split(ts_text["text"]," ")).alias("topic")
    ) \
    .filter(f.col("topic").startswith("#")) \
    .filter(f.col("topic") != "#")

## 3.3 Count Topics

Now that we have the hash tags (topics), we perform a simple aggregation as usual: Group by hashtag (`topic`) and count number of tweets (using `count` or `sum(1)`)

In [17]:
counts = topics \
    .groupBy("topic") \
    .agg(f.sum(f.lit(1)).alias("count"))

In [18]:
counts.printSchema()

root
 |-- topic: string (nullable = false)
 |-- count: long (nullable = true)



## 3.4 Print Results onto Console

Again we want to print the results onto the console.

In [19]:
query = counts.writeStream \
    .format("console") \
    .outputMode("update") \
    .option("truncate", False) \
    .option("checkpointLocation", "/tmp/zeppelin/checkpoint-twitter-count-" + str(time.time())) \
    .start()
    

23/11/02 18:25:41 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/11/02 18:25:41 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+-----+
|topic|count|
+-----+-----+
+-----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+----------------------------------------+-----+
|topic                                   |count|
+----------------------------------------+-----+
|#오피뷰\n♥비아그라/시알리스OIO↔4898↔875O|1    |
|#JumatBerkah                            |1    |
|#재혼만남\n#돌싱카페                    |1    |
|#돌싱카페                               |1    |
|#深夜ラーメン                           |1    |
|#재혼만남                               |1    |
|#一振いちご                             |1    |
|#돌싱카페\n#애인대행                    |1    |
|#애인대행\n#비아그라파는곳              |1    |
|#PJNET                                  |1    |
|#New                                    |1    |
|#BIGOLIVE.                              |1    |
|#애인대행                               |1    |
+----------------------------------------+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+-----+
|top

In [20]:
query.stop()

-------------------------------------------
Batch: 3
-------------------------------------------
+-----------------------+-----+
|topic                  |count|
+-----------------------+-----+
|#発達障害労働を考える会|1    |
+-----------------------+-----+



# 4. Time-Windowed Aggregation

Another interesting (and probably more realistic) application is to perform time windowed aggregations. This means that we define a sliding time window used in the `groupBy` clause. In addition we also define a so called *watermark* which tells Spark how long to wait for late arrivels of individual data points (we don't have them in our simple example).

In [13]:
windowedCounts = topics \
    .withWatermark("timestamp", "10 seconds") \
    .groupBy(f.window(topics["timestamp"], "5 seconds", "1 seconds"), topics["topic"]) \
    .agg(f.sum(f.lit(1)).alias("count"))
    

In [14]:
windowedCounts.printSchema()

root
 |-- window: struct (nullable = true)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- topic: string (nullable = false)
 |-- count: long (nullable = true)



## 4.1 Output Data

Let's again output the data. This time, we also like to investigate the different output modes `append`, `complete` and `update`.

In [27]:
query = windowedCounts.writeStream \
    .outputMode("update") \
    .format("console") \
    .trigger(processingTime="1 seconds") \
    .option("checkpointLocation", "/tmp/zeppelin/checkpoint-twitter-console-" + str(time.time())) \
    .option("truncate", False) \
    .start()   
    

23/11/02 18:28:41 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/11/02 18:28:41 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
23/11/02 18:28:52 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 11048 milliseconds
23/11/02 18:28:57 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 5021 milliseconds
23/11/02 18:28:59 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 1526 milliseconds
23/11/02 18:29:01 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 1976 milliseconds
23/11/02 18:29:05 WARN ProcessingTimeExecutor: Current

In [28]:
query.stop()

23/11/02 18:30:19 WARN SQLAppStatusListener: Unable to load custom metric object for class `org.apache.spark.sql.kafka010.OffsetOutOfRangeMetric`. Please make sure that the custom metric class is in the classpath and it has 0-arg constructor.
java.lang.ClassNotFoundException: org.apache.spark.sql.kafka010.OffsetOutOfRangeMetric
	at java.net.URLClassLoader.findClass(URLClassLoader.java:387) ~[?:1.8.0_382]
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418) ~[?:1.8.0_382]
	at sun.misc.Launcher$AppClassLoader.loadClass(Launcher.java:352) ~[?:1.8.0_382]
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351) ~[?:1.8.0_382]
	at java.lang.Class.forName0(Native Method) ~[?:1.8.0_382]
	at java.lang.Class.forName(Class.java:348) ~[?:1.8.0_382]
	at org.apache.spark.util.Utils$.classForName(Utils.scala:226) ~[spark-core_2.12-3.4.1-amzn-0.jar:3.4.1-amzn-0]
	at org.apache.spark.util.Utils$.$anonfun$loadExtensions$1(Utils.scala:3042) ~[spark-core_2.12-3.4.1-amzn-0.jar:3.4.1-amzn-0]
	at scala

In [16]:
windowedCountsAsValues = windowedCounts.withColumn("value", 
            f.to_json(
                f.struct(
                    windowedCounts["window"],
                    windowedCounts["topic"],
                    windowedCounts["count"]
                )
            )
        )

In [21]:


query = windowedCountsAsValues.writeStream \
    .outputMode("update") \
    .format("kafka") \
    .option("kafka.bootstrap.servers", master) \
    .option("topic", "kku") \
    .trigger(processingTime="1 seconds") \
    .option("checkpointLocation", "/tmp/zeppelin/checkpoint-twitter-console-" + str(time.time())) \
    .start()   

23/11/02 18:38:06 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/11/02 18:38:06 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
23/11/02 18:38:17 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 11145 milliseconds
23/11/02 18:38:22 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 5015 milliseconds
23/11/02 18:38:24 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 2012 milliseconds
23/11/02 18:38:26 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 2093 milliseconds
23/11/02 18:38:28 WARN ProcessingTimeExecutor: Current

/opt/kafka/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic kku

In [22]:
query.stop()

23/11/02 18:39:05 WARN DFSClient: Caught exception 
java.lang.InterruptedException: sleep interrupted
	at java.lang.Thread.sleep(Native Method) ~[?:1.8.0_382]
	at org.apache.hadoop.hdfs.DFSOutputStream.completeFile(DFSOutputStream.java:973) ~[hadoop-client-api-3.3.3-amzn-5.jar:?]
	at org.apache.hadoop.hdfs.DFSOutputStream.completeFile(DFSOutputStream.java:909) ~[hadoop-client-api-3.3.3-amzn-5.jar:?]
	at org.apache.hadoop.hdfs.DFSOutputStream.closeImpl(DFSOutputStream.java:892) ~[hadoop-client-api-3.3.3-amzn-5.jar:?]
	at org.apache.hadoop.hdfs.DFSOutputStream.close(DFSOutputStream.java:847) ~[hadoop-client-api-3.3.3-amzn-5.jar:?]
	at org.apache.hadoop.fs.FSDataOutputStream$PositionCache.close(FSDataOutputStream.java:78) ~[hadoop-client-api-3.3.3-amzn-5.jar:?]
	at org.apache.hadoop.fs.FSDataOutputStream.close(FSDataOutputStream.java:107) ~[hadoop-client-api-3.3.3-amzn-5.jar:?]
	at org.apache.spark.sql.execution.streaming.CheckpointFileManager$RenameBasedFSDataOutputStream.close(Checkpoin